In [25]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.metrics import recall_score, accuracy_score,f1_score, precision_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import optuna
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score


In [26]:
# pip install optuna-dashboard
# optuna-dashboard sqlite:///db.sqlite3

In [27]:
randomState = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
raw_dataset = pd.read_csv("./data/processed_data.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])

#* 90/10 split for training and final test
X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=randomState, stratify=Y)

Using device: cpu


In [28]:
def FOLDS_GENERATOR(X, Y, normalisation_method=MinMaxScaler(), n_splits=5, randomState=None, oversample=False):
    
    """
    Generates stratified folds with specified normalization.
    
    For list of scalers, see:
    https://scikit-learn.org/stable/api/sklearn.preprocessing.html
    
    For more details on scaling and normalization effects, see:
    https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#
    
    normalisation_method should be an instance of a scaler, e.g.,
    - MinMaxScaler()
    - MaxAbsScaler()
    - Quantile_Transform(output_distribution='uniform')
    
    Returns a list of tuples, each containing:
    (X_train_scaled, X_test_scaled, Y_train, Y_test), representing data for each fold
    """
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=randomState)
    kFolds_list = []
    
    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
        
        # Fit the scaler on the training data and transform both train and test sets
        X_train_scaled = normalisation_method.fit_transform(X_train)
        X_test_scaled = normalisation_method.transform(X_test)
        
        if oversample:
            # Oversample the training data if needed (e.g., using SMOTE or similar techniques)
            # This is a placeholder; actual oversampling code should be implemented here
            # X_train_scaled....
            pass
        
        # Convert back to DataFrame to maintain column names
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)
        
        # Ensure 'gender' is still binary (0 or 1)
        if X_train_scaled['Gender'].isin([0, 1]).all():
            kFolds_list.append((X_train_scaled, X_test_scaled, Y_train, Y_test))
        else:
            print("Warning: 'gender' column contains unexpected values after scaling.") 
               
        print(f"Fold: {fold+1}, Train: {kFolds_list[fold][0].shape}, Test: {kFolds_list[fold][1].shape}")   
    return kFolds_list

def init_weights(model): #tested already
    if isinstance(model, nn.Linear):  # Apply only to linear layers
        nn.init.xavier_uniform_(model.weight)
        if model.bias is not None:
            nn.init.zeros_(model.bias)
            
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batchSize=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
    return train_loader, val_loader 

In [29]:
kFolds = FOLDS_GENERATOR(X_FOR_FOLDS, Y_FOR_FOLDS, normalisation_method=MinMaxScaler(), n_splits=5, randomState=randomState)

Fold: 1, Train: (4593, 28), Test: (1149, 28)
Fold: 2, Train: (4593, 28), Test: (1149, 28)
Fold: 3, Train: (4594, 28), Test: (1148, 28)
Fold: 4, Train: (4594, 28), Test: (1148, 28)
Fold: 5, Train: (4594, 28), Test: (1148, 28)


In [30]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [31]:
from Criterion_Models import *
def criterion_mapping(criterion_choice:str, pos_weight:float=None):
    """
    Feel free to add any custom loss functions here.
    returns function for criterion
    """
    if criterion_choice == "FocalLoss":
        return FocalLoss()
    elif criterion_choice == "DiceLoss":
        return DiceLoss()
    elif criterion_choice == "BCEWithLogitsLoss":
        return nn.BCEWithLogitsLoss(pos_weight=torch.tensor([pos_weight])) if pos_weight else nn.BCEWithLogitsLoss()
    return nn.BCEWithLogitsLoss() 

In [32]:
def train_and_evaluate(model, criterion, optimiser, scheduler, kFolds, epochs, batchsize, patience=5, device=device):
    if isinstance(model[-1], nn.Sigmoid) and isinstance(criterion, nn.BCEWithLogitsLoss):
        raise ValueError("Model output is Sigmoid but criterion is BCEWithLogitsLoss. Please check your model and criterion compatibility.")

    
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    auc_list = []
    
    for fold, (train_x, test_x, train_y, test_y) in enumerate(kFolds, start=1):
        best_val_loss = float('inf')
        best_model_state = None
        wait = 0
        # print(f"Fold: {fold}")
        #* Convert the fold to PyTorch tensors and create DataLoader objects
        train_loader, val_loader = fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batchsize, device)

        #* Set model to training mode: essential for dropout and batch norm layers
        model.train()
        #* Epoch Training loop for this fold
        for epoch in range(1,epochs+1):
            running_loss = 0.0 #? loss for this epoch
            #* Mini-batch training loop
            for batch, (inputs, labels) in enumerate(train_loader,start=1):
                optimiser.zero_grad() #? Zero the gradients
                outputs = model(inputs) #? Forward pass through the model
                loss = criterion(outputs, labels) #? Calculate loss
                loss.backward() #? Backpropagation
                running_loss += loss.item()
                optimiser.step() #? Update weights
                if scheduler:
                    scheduler.step()
                   
            train_loss = running_loss / len(train_loader)
            print(f"Epoch: {epoch}, training loss: {train_loss:.4f}")
        
            #* Now we evaluate the model on the validation set, to track training vs validation loss
            model.eval() #? Set model to evaluation mode
            with torch.no_grad(): #? No need to track gradients during evaluation
                val_loss = 0.0    
                for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                    outputs = model(inputs)
                    if isinstance(model[-1], nn.Sigmoid):
                        predictions = (outputs > 0.5).float().cpu() #? assume model output is 1s and 0s
                    else: #? if model output is logits, convert to binary predictions
                        predictions = (torch.sigmoid(outputs) > 0.5).float().cpu()
                    labels = labels.cpu() 
                    loss = criterion(predictions, labels)
                    val_loss += loss.item() #? Calculate loss
                    avg_val_loss = val_loss / len(val_loader)
                    print(f"Epoch {epoch}, Val Loss: {avg_val_loss:.4f}")
            
                    # Early stopping
                    if avg_val_loss < best_val_loss:
                        best_val_loss = avg_val_loss
                        best_model_state = model.state_dict()
                        wait = 0
                    else:
                        wait += 1
                        if wait >= patience:
                            print(f"Early stopping triggered at epoch {epoch+1}")
                            break
        
        #* Use best model to calculate metrics on the validation set
        #! must be outside epoch loop, it comes after the training and cv loop
        model.load_state_dict(best_model_state) #? Load the best model state
        with torch.nograd():
            for batch, (inputs, labels) in enumerate(val_loader,start=1):#! one pass because val_loader batch size is all, if you want to do it in mini-batches, you MUST change the metric calculations to accept mini-batches
                    outputs = model(inputs)
                    if isinstance(model[-1], nn.Sigmoid):
                        predictions = (outputs > 0.5).float().cpu() #? assume model output is 1s and 0s
                    else: #? if model output is logits, convert to binary predictions
                        predictions = (torch.sigmoid(outputs) > 0.5).float().cpu()
                    labels = labels.cpu() 
                    loss = criterion(predictions, labels)
                    val_loss += loss.item() #? Calculate loss
                    
        #! The following should have length equal to fold number           
        accuracy_list.append(accuracy_score(labels, predictions)) 
        precision_list.append(precision_score(labels, predictions, pos_label=1)) 
        recall_list.append(recall_score(labels, predictions, pos_label=1))
        f1_list.append(f1_score(labels, predictions, pos_label=1))
        auc_list.append(roc_auc_score(labels, predictions)) 

    return model, accuracy_list, precision_list, recall_list, f1_list, auc_list 


In [ ]:
def objective(trial):
    #* Model hyperparameters (first-level optimization)
    hidden_dim = trial.suggest_int("hidden_dim", 16, 128)  #? Number of units in the hidden layer
    dropout = trial.suggest_float("dropout", 0.1, 0.5)     #? Dropout rate for regularization
    initial_lr = trial.suggest_float("initial_lr", 1e-5, 1e-3, log=True) #? Initial learning rate for the optimizer
    max_lr = trial.suggest_loguniform("max_lr", 1e-3, 1e-1)
    #? Learning rate on a log scale
    
    #* Loss function hyperparameters (first-level optimization)
    criterion_choice = trial.suggest_categorical("criterion", ["BCEWithLogitsLoss", "FocalLoss", "DiceLoss"]) 
    #? See which loss functions yield the best results
    
    #* Hyperparameter exploration optimization(second-level optimization)
    if isinstance(criterion_choice, nn.BCEWithLogitsLoss): 
        #? For BCEWithLogitsLoss: see which weight gives the best results
        pos_weight = trial.suggest_int("pos_weight", 1.0, 10.0) 
    else:
        pos_weight = None
    # if criterion_choice == ..
        
    #* Loading in the hyperparameters for training
    model = BinaryClassifier(input_dim=20, hidden_dim=hidden_dim, dropout=dropout)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.apply(init_weights)
    
    #* Map the choice to the actual loss function
    criterion = criterion_mapping(criterion_choice, pos_weight).to(device) #! always .to(device) this
    optimiser = optim.Adam(model.parameters(), lr=max_lr)
    
    
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimiser,
        max_lr=max_lr,
        steps_per_epoch=len(train_loader),
        epochs=epoch,
        anneal_strategy='linear'
    )
    
    model, accuracy_list, precision_list, recall_list, f1_list, auc_list = train_and_evaluate(model, criterion, optimiser, scheduler, kFolds, epochs=100, batchsize=64, patience=5, device=device)

    # Calculate the average of each metric across all folds
    avg_accuracy = sum(accuracy_list) / len(accuracy_list)
    avg_precision = sum(precision_list) / len(precision_list)
    avg_recall = sum(recall_list) / len(recall_list)
    avg_f1 = sum(f1_list) / len(f1_list)
    avg_auc = sum(auc_list) / len(auc_list)

    # Combine metrics into a single "score" (simple average here, but feel free to adjust the weights)
    combined_score = (avg_f1 + avg_precision + avg_recall + avg_accuracy + avg_auc) / 5  # or use weighted sum

    return combined_score


In [38]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # You can adjust the number of trials

print("Best trial:")
trial = study.best_trial
print(f"  F1 Score: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-04-05 23:54:21,625] A new study created in memory with name: no-name-1d89d88d-54b0-4843-bafb-c11002bb15c8
C:\Users\tanle\AppData\Local\Temp\ipykernel_10640\1161310067.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  max_lr = trial.suggest_loguniform("max_lr", 1e-3, 1e-1)
[W 2025-04-05 23:54:28,264] Trial 0 failed with parameters: {'hidden_dim': 65, 'dropout': 0.3917573210540152, 'initial_lr': 0.00020480352771089, 'max_lr': 0.09374634214340735, 'criterion': 'FocalLoss'} because of the following error: NameError("name 'train_loader' is not defined").
Traceback (most recent call last):
  File "d:\GitHub repos\ADL\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\tanle\AppData\Local\Temp\ipykernel_10640\11613100

NameError: name 'train_loader' is not defined